In [1]:
from weavepy import *
sns.set_context("talk")
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import plotly.io as pio
#pio.renderers.default = "notebook_connected"

####################################################################################################################################################
################################################################## --- WIDGETS --- #################################################################
####################################################################################################################################################

##-------------------------##
## --- CLIMATE Widgets --- ##
##-------------------------##

wdg_climate_variable = widgets.Dropdown(
    options={clim_vars['name']: key for key, clim_vars in CLIM_VARS_DICT.items()},
    value="TA",
    )
climate_variable_label = widgets.HTML(value="<b>Climate Variable:</b>")

##-------------------------##
## --- ENERGY  Widgets --- ##
##-------------------------##

wdg_energy_variable = widgets.Dropdown(
    options={names: key for key, names in ENER_VARS_DICT.items()},
    value="SPV",
    )
energy_variable_label = widgets.HTML(value="<b>Energy Variable:</b>")

wdg_techno = widgets.Dropdown(
    options={f'{name} ({key})' : key for key, name in ENER_VARS_TECHNOS[wdg_energy_variable.value].items()},
    )
techno_variable_label = widgets.HTML(value="Technology:")

##--------------------------------------##
## --- AGGREGATION & ROLLING WINDOW --- ##
##--------------------------------------##

wdg_aggregation_frequency = widgets.Dropdown(
    options={name : key for key, name in FREQUENCIES_DICT.items()},
    value="D",
)
aggregation_frequency_label = widgets.HTML(value="<b>Aggregation Frequency:</b>")

wdg_aggregation_function = widgets.Dropdown(
    options=["mean", "sum", "max", "min"],
    value="mean",
    )
aggregation_function_label = widgets.HTML(value="<b>Aggregation Function:</b>")

wdg_rolling_window_selector = widgets.IntSlider(
    min=1, max=60, step=1, value=20,
)
rolling_window_selector_label = widgets.HTML(value="<b>Rolling Window</b> (number of years over which averaging time series):")

##-------------------------##
## --- EVENT DETECTION --- ##
##-------------------------##
clim_event_def_txt = widgets.HTML(value="<b>Define climate event:</b>")

wdg_climate_variable_comparison = widgets.RadioButtons(
    options=["<", "<=", ">", ">="],
    value="<=",
)
climate_variable_comparison_label = widgets.HTML(value="Climate Operator:")

wdg_climate_variable_threshold = widgets.FloatSlider(
    min=CLIM_VARS_DICT[wdg_climate_variable.value]['min'],
    max=CLIM_VARS_DICT[wdg_climate_variable.value]['max'],
    step=CLIM_VARS_DICT[wdg_climate_variable.value]['step'],
    value=CLIM_VARS_DICT[wdg_climate_variable.value]['default'],
)
climate_variable_threshold_label = widgets.HTML(value="Climate Threshold:")
climate_variable_threshold_unit = widgets.HTML(value=f"<b>{CLIM_VARS_DICT[wdg_climate_variable.value]['units']}</b>")

energy_event_def_txt = widgets.HTML(value="<b>Define energy event:</b>")

wdg_energy_variable_comparison = widgets.RadioButtons(
    options=["<", "<=", ">", ">="],
    value="<",
)
energy_variable_comparison_label = widgets.HTML(value="Energy Operator:")

wdg_energy_variable_threshold = widgets.FloatSlider(
    min=0, max=100, step=1, value=5,
)
energy_variable_threshold_label = widgets.HTML(value="Energy Threshold (capacity factor):")
energy_variable_threshold_unit = widgets.HTML(value="<b>%</b>")

##-------------------------##
## ---  TIME  PERIODS  --- ##
##-------------------------##

wdg_historical_period = widgets.IntRangeSlider(
    value=[1990, 2025],
    min=1950, max=2025, step=1,
    )
historical_period_label = widgets.HTML(value="<b>Historical period:</b>")

wdg_future_period = widgets.IntRangeSlider(
    value=[2015, 2100],
    min=2015, max=2100, step=1,
    )
future_period_label = widgets.HTML(value="<b>Future period:</b>")

##-------------------------##
## -- COUNTRY SELECTION -- ##
##-------------------------##

wdg_country = widgets.Dropdown(
    options={names: key for key, names in COUNTRIES_DICT.items()},
    )
country_label = widgets.HTML(value="<b>Country:</b>")

##-------------------------##
## -- MODELS  SELECTION -- ##
##-------------------------##
models_txt = widgets.HTML(value="<b>Select models used to compute the multi-model mean:</b>")
scenarios_txt = widgets.HTML(value="<b>Note:</b> Scenario selection/deselection is possible directly on the plots after execution of the program.")

wdg_models = MODEL_NAMES

# Individual checkboxes
model_checkboxes = {name: widgets.Checkbox(value=True, description=name) for name in MODEL_NAMES}

# Select/Deselect All checkbox
chk_all_models = widgets.Checkbox(value=True, description="Select/Deselect All models", indent=False)

##-------------------------##
## - SCENARIOS SELECTION - ##
##-------------------------##

wdg_scenarios = SCENARIOS

# Individual checkboxes
scenarios_checkboxes = {name: widgets.Checkbox(value=True, description=name) for name in SCENARIOS}

# Select/Deselect All checkbox
chk_all_scenarios = widgets.Checkbox(value=True, description="Select/Deselect All scenarios", indent=False)

##-------------------------##
## -- EXECUTION WIDGETS -- ##
##-------------------------##

status_label = widgets.Label(value="Ready.")
execute_button = widgets.Button(description="Execute", button_style="success")

####################################################################################################################################################
################################################################# --- FUNCTIONS --- ################################################################
####################################################################################################################################################

##-------------------------##
## -- MODELS  SELECTION -- ##
##-------------------------##

def sync_models_checkboxes(change=None):
    """Update chk_all_models to reflect current state of model_checkboxes."""
    if all(chk.value for chk in model_checkboxes.values()):
        chk_all_models.unobserve(toggle_all_models, "value")  # prevent recursion
        chk_all_models.value = True
        chk_all_models.observe(toggle_all_models, "value")
    else:
        chk_all_models.unobserve(toggle_all_models, "value")
        chk_all_models.value = False
        chk_all_models.observe(toggle_all_models, "value")

def update_models_list(change=None):
    global wdg_models
    wdg_models = [name for name, chk in model_checkboxes.items() if chk.value]

def toggle_all_models(change):
    """Select/deselect all checkboxes when chk_all_models is toggled."""
    if change["new"] is True:
        for chk in model_checkboxes.values():
            chk.value = True
    elif change["new"] is False:
        for chk in model_checkboxes.values():
            chk.value = False

##-------------------------##
## - SCENARIOS SELECTION - ##
##-------------------------##

def sync_scenarios_checkboxes(change=None):
    """Update chk_all_scenarios to reflect current state of scenarios_checkboxes."""
    if all(chk.value for chk in scenarios_checkboxes.values()):
        chk_all_scenarios.unobserve(toggle_all_scenarios, "value")  # prevent recursion
        chk_all_scenarios.value = True
        chk_all_scenarios.observe(toggle_all_scenarios, "value")
    else:
        chk_all_scenarios.unobserve(toggle_all_scenarios, "value")
        chk_all_scenarios.value = False
        chk_all_scenarios.observe(toggle_all_scenarios, "value")

def update_scenarios_list(change=None):
    global wdg_scenarios
    wdg_scenarios = [name for name, chk in scenarios_checkboxes.items() if chk.value]

def toggle_all_scenarios(change):
    """Select/deselect all checkboxes when chk_all_scenarios is toggled."""
    if change["new"] is True:
        for chk in scenarios_checkboxes.values():
            chk.value = True
    elif change["new"] is False:
        for chk in scenarios_checkboxes.values():
            chk.value = False

##-------------------------##
## --- ENER VAR TECHNO --- ##
##-------------------------##

def update_techno(change):
    new_energy = change["new"]
    wdg_techno.options = {f'{name} ({key})' : key for key, name in ENER_VARS_TECHNOS[new_energy].items()}

##-------------------------##
## --- CLIM  THRESHOLD --- ##
##-------------------------##

def update_climate_threshold_slider(change):
    new_climate_variable = change["new"]
    wdg_climate_variable_threshold.min=CLIM_VARS_DICT[new_climate_variable]['min']
    wdg_climate_variable_threshold.max=CLIM_VARS_DICT[new_climate_variable]['max']
    wdg_climate_variable_threshold.step=CLIM_VARS_DICT[new_climate_variable]['step']
    wdg_climate_variable_threshold.value=CLIM_VARS_DICT[new_climate_variable]['default']
    climate_variable_threshold_unit.value = f"<b>{CLIM_VARS_DICT[new_climate_variable]['units']}</b>"

##-------------------------##
##- SELECTION CHANGED MSG -##
##-------------------------##

def mark_dirty(change):
    global status_label
    status_label.value = "⚠️ Parameters changed. Click Execute to update."

####################################################################################################################################################
################################################################# --- OBSERVERS --- ################################################################
####################################################################################################################################################

##-------------------------##
## ---- BASIC WIDGETS ---- ##
##-------------------------##

for w in [
    wdg_climate_variable, wdg_aggregation_frequency, wdg_aggregation_function,
    wdg_energy_variable, wdg_techno, wdg_climate_variable_comparison, wdg_climate_variable_threshold,
    wdg_energy_variable_comparison, wdg_energy_variable_threshold, wdg_country,
    wdg_historical_period, wdg_future_period, wdg_rolling_window_selector
]:
    w.observe(mark_dirty, names="value")

##-------------------------##
## --- MODELS  WIDGETS --- ##
##-------------------------##

for chk in model_checkboxes.values():
    chk.observe(sync_models_checkboxes, "value")
    chk.observe(update_models_list, "value")
    chk.observe(mark_dirty, names="value")

chk_all_models.observe(toggle_all_models, "value")
chk_all_models.observe(mark_dirty, names="value")

##-------------------------##
## -- SCENARIOS WIDGETS -- ##
##-------------------------##

for chk in scenarios_checkboxes.values():
    chk.observe(sync_scenarios_checkboxes, "value")
    chk.observe(update_scenarios_list, "value")
    chk.observe(mark_dirty, names="value")

chk_all_scenarios.observe(toggle_all_scenarios, "value")
chk_all_scenarios.observe(mark_dirty, names="value")

##-------------------------##
## --- ENER VAR TECHNO --- ##
##-------------------------##

wdg_energy_variable.observe(update_techno, names="value")

##-------------------------##
## --- CLIM  THRESHOLD --- ##
##-------------------------##

wdg_climate_variable.observe(update_climate_threshold_slider, names='value')

####################################################################################################################################################
################################################################# --- EXECUTION --- ################################################################
####################################################################################################################################################

def on_execute_clicked(button):
    status_label.value = "⏳ Running..."

    # Retrieve values from widgets
    variables = [wdg_climate_variable.value, wdg_energy_variable.value]
    aggregation_frequency = wdg_aggregation_frequency.value
    aggregation_function = wdg_aggregation_function.value
    techno = wdg_techno.value
    comparisons = {
                    wdg_climate_variable.value : wdg_climate_variable_comparison.value, 
                    wdg_energy_variable.value : wdg_energy_variable_comparison.value,
                }
    thresholds = {
                    wdg_climate_variable.value : wdg_climate_variable_threshold.value,
                    wdg_energy_variable.value : wdg_energy_variable_threshold.value / 100,  # Convert percentage to a fraction
                }
    country = wdg_country.value
    historical_period = wdg_historical_period.value
    future_period = wdg_future_period.value
    models = wdg_models
    scenarios = wdg_scenarios
    rolling_window = wdg_rolling_window_selector.value
    
    # LOAD Data
    data = load_vars(variables, bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True).sel(model=models,scenario=scenarios)
    
    # Identify problematic days
    days = {}
    for var in variables:
        days[var] = identify_pb_days(data[var], comparisons[var], thresholds[var])
    days["compound"] = xr.concat(list(days.values()), dim = "var").all(dim = "var").rename("compound")
    # Identify events
    events = {}
    for var in variables + ["compound"]:
        events[var] = identify_events_whole_base(days[var])

    with output:
        output.clear_output(wait=True)
        fig0 = event_count_barplot_multi(list(events.values()), historical_period, future_period)
        fig1 = nb_event_timeseries_multi(list(events.values()), rolling_window=rolling_window)
        fig2 = event_duration_hist_multi(list(events.values()), historical_period, future_period)
        fig3 = event_seasonality_kde_multi(list(days.values()), historical_period, future_period)
        display(fig0, fig1, fig2, fig3)

    #Update status label
    status_label.value = "✅ Done!"
    plotly_note.value = "<p><strong>About the Plotly plots</strong></p>\
                         <p>We use plotly to display the figures, which allows user interactions with the plots. In particular, you can:</p>\
                         <ul>\
                         <li>Hover over the plot to show the values for points, lines, or bars.</li>\
                         <li>Click on legend entries to hide/show the corresponding traces.</li>\
                         <li>Double-click a legend entry to isolate a given trace.</li>\
                         </ul>"

plotly_note = widgets.HTML(value="")
output = widgets.Output()
execute_button.on_click(on_execute_clicked)

####################################################################################################################################################
################################################################## --- LAYOUT --- ##################################################################
####################################################################################################################################################

# -- DESING FEATURE -- #
# Add a horizontal line and vertical line for visual separation
hr = widgets.Box(layout=widgets.Layout(border='1px solid #888', width='100%', height='0px'))
vr = widgets.Box(layout=widgets.Layout(border='1px solid #888', width='0px', height='100%'))
# -- MAIN UI CONTAINER -- #
ui = widgets.VBox([
    
        widgets.HBox([

#CLIMATE and ENERGY VARIABLES
            widgets.VBox([climate_variable_label,wdg_climate_variable],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")
                         ),
            widgets.VBox([energy_variable_label, wdg_energy_variable, techno_variable_label, wdg_techno],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")
                         ),
                ]),

#AGGREGATION FREQUENCY and FUNCTION
        widgets.HBox([
            widgets.VBox([aggregation_frequency_label, wdg_aggregation_frequency],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")
                         ),
            widgets.VBox([aggregation_function_label, wdg_aggregation_function],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")
                         ),
                ]),
        hr,

#COUNTRY SELECTION
        widgets.VBox([country_label,wdg_country],
                     layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")),
        hr,

#TIME PERIODS
    widgets.HBox([
        widgets.VBox([historical_period_label, wdg_historical_period,],
                     layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="15px")),
        widgets.VBox([future_period_label, wdg_future_period],
                     layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="15px")),
        widgets.VBox([rolling_window_selector_label, wdg_rolling_window_selector],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")),
        ],
        layout=widgets.Layout(justify_content="center", align_items="center", padding="10px")),
    hr,

#CLIMATE EVENT DETECTION
    widgets.VBox([clim_event_def_txt,
        widgets.HBox([
            widgets.VBox([climate_variable_comparison_label, wdg_climate_variable_comparison],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="5px")),
            widgets.VBox([climate_variable_threshold_label,
                          widgets.HBox([wdg_climate_variable_threshold, climate_variable_threshold_unit])],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="5px"))
            ],
            layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="0px")),
        ],
        layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")),
    hr,
#ENERGY EVENT DETECTION
    widgets.VBox([energy_event_def_txt,
        widgets.HBox([
            widgets.VBox([energy_variable_comparison_label, wdg_energy_variable_comparison],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="5px")),
            widgets.VBox([energy_variable_threshold_label,
                          widgets.HBox([wdg_energy_variable_threshold, energy_variable_threshold_unit])],
                         layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="5px"))
            ],
            layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="0px")),
        ],
        layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")),
    hr,
#MODELS and SCENARIOS SELECTION
    widgets.VBox([models_txt,
        widgets.HBox([
            widgets.VBox([chk_all_models] + list(model_checkboxes.values())),
    #        widgets.VBox([chk_all_scenarios] + list(scenarios_checkboxes.values())),
        ]),
        scenarios_txt
    ],
    layout=widgets.Layout(justify_content="flex-start", align_items="flex-start", padding="10px")),
    hr,
#EXECUTION BUTTON
    execute_button

],layout=widgets.Layout(justify_content="space-between", align_items="center", border="6px solid #888", padding="5px"))

display(widgets.VBox([ui, status_label, plotly_note, output]))
